In [435]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import joblib

In [436]:
import pandas as pd

# We'll also import seaborn, a Python graphing library
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)


data=pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [437]:
diabetes_data_copy = data.copy(deep = True)
diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.nan)

## showing the count of Nans
print(diabetes_data_copy.isnull().sum())



diabetes_data_copy['Glucose'].fillna(diabetes_data_copy['Glucose'].mean(), inplace = True)
diabetes_data_copy['BloodPressure'].fillna(diabetes_data_copy['BloodPressure'].mean(), inplace = True)
diabetes_data_copy['SkinThickness'].fillna(diabetes_data_copy['SkinThickness'].median(), inplace = True)
diabetes_data_copy['Insulin'].fillna(diabetes_data_copy['Insulin'].median(), inplace = True)
diabetes_data_copy['BMI'].fillna(diabetes_data_copy['BMI'].median(), inplace = True)



Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


# 1. MLP

In [ ]:
# Gerekli kütüphaneleri import edelim
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# 1. Split FIRST (to avoid data leakage)
X = diabetes_data_copy.drop("Outcome", axis=1)
y = diabetes_data_copy["Outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Initialize and fit scaler ONLY on training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit only on train
X_test_scaled = scaler.transform(X_test)        # Transform test data

joblib.dump(scaler, 'scaler.pkl')

# 3. Create and train MLP
mlp = MLPClassifier(
    hidden_layer_sizes=(50, 50),
    max_iter=5000,
    alpha=0.0001,
    activation='relu',
    random_state=42,
    early_stopping=True,  # Add early stopping
    validation_fraction=0.1  # Use 10% of training for validation
)

# 4. Train the model
mlp.fit(X_train_scaled, y_train)

# 5. Evaluate
predictions = mlp.predict(X_test_scaled)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, predictions))
print("\n=== Classification Report ===")
print(classification_report(y_test, predictions))
print(f"\nAccuracy: {accuracy_score(y_test, predictions):.2f}")

# 6. Additional useful outputs
print("\nTraining set score: %.2f" % mlp.score(X_train_scaled, y_train))
print("Test set score: %.2f" % mlp.score(X_test_scaled, y_test))

=== Confusion Matrix ===
[[148   3]
 [ 77   3]]

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.66      0.98      0.79       151
           1       0.50      0.04      0.07        80

    accuracy                           0.65       231
   macro avg       0.58      0.51      0.43       231
weighted avg       0.60      0.65      0.54       231


Accuracy: 0.65

Training set score: 0.65
Test set score: 0.65


## 1.1. MLP (PCA)

In [ ]:
from sklearn.decomposition import PCA


# Apply PCA
n_components = 1  # Number of principal components to keep
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

joblib.dump(pca, 'pca.pkl')

print("Original data shape:", X.shape)
print("Reduced data shape:", X_train_pca.shape)
print("Explained variance ratio:", pca.explained_variance_ratio_)


# MLP modelini oluşturalım
mlp = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=5000, random_state=42)
# hidden_layer_sizes: Her bir tuple elemanı bir gizli katmanı ve nöron sayısını temsil eder
# Bu örnekte 3 gizli katman, her birinde 10 nöron var

# Modeli eğitelim
mlp.fit(X_train_pca, y_train)

# Tahminler yapalım
predictions = mlp.predict(X_test_pca)

# Confusion Matrix
cm = confusion_matrix(y_test, predictions)

tn, fp, fn, tp = cm.ravel()

# Calculate specificity
specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

from sklearn.metrics import recall_score
sensitivity = recall_score(y_test, predictions)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

from sklearn.metrics import precision_score
precision = precision_score(y_test, predictions)
print(f"Kesinlik (Precision): {precision:.4f}")

from sklearn.metrics import f1_score
f1 = f1_score(y_test, predictions)
print(f"F1 Skoru: {f1:.4f}")

Original data shape: (768, 8)
Reduced data shape: (537, 1)
Explained variance ratio: [0.2824466]
Specificity: 0.7815
Doğruluk (Accuracy): 0.6840
Duyarlılık (Sensitivity/Recall): 0.5000
Kesinlik (Precision): 0.5479
F1 Skoru: 0.5229


## 1.3. MLP (LDA)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
import numpy as np


# Apply LDA
n_components = 1  # Number of components to keep (cannot be greater than n_classes - 1)
lda = LinearDiscriminantAnalysis(n_components=n_components)
X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

joblib.dump(lda, 'lda.pkl')

print("Original data shape:", X.shape)
print("Reduced data shape:", X_train_lda.shape)

# MLP modelini oluşturalım
mlp = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=5000, random_state=42)
# hidden_layer_sizes: Her bir tuple elemanı bir gizli katmanı ve nöron sayısını temsil eder
# Bu örnekte 3 gizli katman, her birinde 10 nöron var

# Modeli eğitelim
mlp.fit(X_train_lda, y_train)

# Tahminler yapalım
predictions = mlp.predict(X_test_lda)

# Confusion Matrix
cm = confusion_matrix(y_test, predictions)

tn, fp, fn, tp = cm.ravel()

# Calculate specificity
specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

from sklearn.metrics import recall_score
sensitivity = recall_score(y_test, predictions)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

from sklearn.metrics import precision_score
precision = precision_score(y_test, predictions)
print(f"Kesinlik (Precision): {precision:.4f}")

from sklearn.metrics import f1_score
f1 = f1_score(y_test, predictions)
print(f"F1 Skoru: {f1:.4f}")

Original data shape: (768, 8)
Reduced data shape: (537, 1)
Specificity: 0.7815
Doğruluk (Accuracy): 0.7273
Duyarlılık (Sensitivity/Recall): 0.6250
Kesinlik (Precision): 0.6024
F1 Skoru: 0.6135


# 2. Decision Tree

In [441]:
from sklearn.tree import DecisionTreeClassifier

dt= DecisionTreeClassifier()
dt.fit(X_train_scaled,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_scaled)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.6145
Doğruluk (Accuracy): 0.7013
Kesinlik (Precision): 0.5556
Duyarlılık (Sensitivity/Recall): 0.6875
Specificity: 0.7086


## 2.1. Decision Tree (PCA)

In [442]:

from sklearn.tree import DecisionTreeClassifier

dt= DecisionTreeClassifier()
dt.fit(X_train_pca,y_train)


# Make predictions on the testing set
y_pred = dt.predict(X_test_pca)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.5000
Doğruluk (Accuracy): 0.6364
Kesinlik (Precision): 0.4773
Duyarlılık (Sensitivity/Recall): 0.5250
Specificity: 0.6954


## 2.3. Decision Tree (LDA)

In [443]:

from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier()
dt.fit(X_train_lda,y_train)


# Make predictions on the testing set
y_pred = dt.predict(X_test_lda)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.6154
Doğruluk (Accuracy): 0.7186
Kesinlik (Precision): 0.5843
Duyarlılık (Sensitivity/Recall): 0.6500
Specificity: 0.7550


# 3. K-NEAREST NEIGHBOR

In [444]:

from sklearn.neighbors import KNeighborsClassifier
dt = KNeighborsClassifier(n_neighbors=1)
dt.fit(X_train_scaled,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_scaled)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.5389
Doğruluk (Accuracy): 0.6667
Kesinlik (Precision): 0.5172
Duyarlılık (Sensitivity/Recall): 0.5625
Specificity: 0.7219


## 3.1. K-NEAREST NEIGHBOOR (PCA)

In [445]:
from sklearn.neighbors import KNeighborsClassifier
dt = KNeighborsClassifier(n_neighbors=1)
dt.fit(X_train_pca,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_pca)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.5000
Doğruluk (Accuracy): 0.6364
Kesinlik (Precision): 0.4773
Duyarlılık (Sensitivity/Recall): 0.5250
Specificity: 0.6954


## 3.1. K-NEAREST NEIGHBOOR (LDA)

In [446]:
from sklearn.neighbors import KNeighborsClassifier
dt = KNeighborsClassifier(n_neighbors=1)
dt.fit(X_train_lda,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_lda)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.6154
Doğruluk (Accuracy): 0.7186
Kesinlik (Precision): 0.5843
Duyarlılık (Sensitivity/Recall): 0.6500
Specificity: 0.7550


# 4. NAIVE BAYES

In [447]:

from sklearn.naive_bayes import BernoulliNB
dt = BernoulliNB()
dt.fit(X_train_scaled,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_scaled)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.5795
Doğruluk (Accuracy): 0.6797
Kesinlik (Precision): 0.5312
Duyarlılık (Sensitivity/Recall): 0.6375
Specificity: 0.7020


## 4.1. NAIVE BAYES (PCA)

In [448]:

from sklearn.naive_bayes import BernoulliNB
dt = BernoulliNB()
dt.fit(X_train_pca,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_pca)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

joblib.dump(dt, "bayes_pca_model.pkl")

F1 Skoru: 0.6373
Doğruluk (Accuracy): 0.6797
Kesinlik (Precision): 0.5242
Duyarlılık (Sensitivity/Recall): 0.8125
Specificity: 0.6093


['bayes_pca_model.pkl']

## 4.3. NAIVE BAYES (LDA)

In [449]:

from sklearn.naive_bayes import BernoulliNB
dt = BernoulliNB()
dt.fit(X_train_lda,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_lda)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

joblib.dump(dt, "bayes_lda_model.pkl")


F1 Skoru: 0.6178
Doğruluk (Accuracy): 0.6840
Kesinlik (Precision): 0.5315
Duyarlılık (Sensitivity/Recall): 0.7375
Specificity: 0.6556


['bayes_lda_model.pkl']

# 5. SUPPORT VECTOR MACHINE

In [450]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd

# 4. Initialize and Train SVC Model
model = SVC(random_state=42)
model.fit(X_train_scaled, y_train)

# 5. Make Predictions
y_pred = model.predict(X_test_scaled)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.6000
Doğruluk (Accuracy): 0.7403
Kesinlik (Precision): 0.6429
Duyarlılık (Sensitivity/Recall): 0.5625
Specificity: 0.8344


## 5.1. SUPPORT VECTOR MACHINE (PCA)

In [451]:

# 4. Initialize and Train SVC Model
model = SVC(random_state=42)
model.fit(X_train_pca, y_train)

# 5. Make Predictions
y_pred = model.predict(X_test_pca)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.5235
Doğruluk (Accuracy): 0.6926
Kesinlik (Precision): 0.5652
Duyarlılık (Sensitivity/Recall): 0.4875
Specificity: 0.8013


## 5.3. SUPPORT VECTOR MACHINE (LDA)

In [452]:

# 4. Initialize and Train SVC Model
model = SVC(kernel='linear', random_state=42)
model.fit(X_train_lda, y_train)

# 5. Make Predictions
y_pred = model.predict(X_test_lda)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

joblib.dump(model, "svm_lda_model.pkl")

F1 Skoru: 0.6144
Doğruluk (Accuracy): 0.7446
Kesinlik (Precision): 0.6438
Duyarlılık (Sensitivity/Recall): 0.5875
Specificity: 0.8278


['svm_lda_model.pkl']

# 6. LOGISTIC REGRESSION

In [453]:
from sklearn.utils import class_weight

# Class weight hesapla
classes_weights = class_weight.compute_class_weight('balanced', 
                                                 classes=np.unique(y_train), 
                                                 y=y_train)
weights = {0: classes_weights[0], 1: classes_weights[1]}

from sklearn.linear_model import LogisticRegression
dt = LogisticRegression(random_state=0, class_weight=weights)
dt.fit(X_train_scaled,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_scaled)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

joblib.dump(dt, "lr_model.pkl")

F1 Skoru: 0.6092
Doğruluk (Accuracy): 0.7056
Kesinlik (Precision): 0.5638
Duyarlılık (Sensitivity/Recall): 0.6625
Specificity: 0.7285


['lr_model.pkl']

## 6.1. LOGISTIC REGRESSION (PCA)

In [454]:

dt = LogisticRegression(random_state=0)
dt.fit(X_train_pca,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_pca)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

F1 Skoru: 0.5200
Doğruluk (Accuracy): 0.6883
Kesinlik (Precision): 0.5571
Duyarlılık (Sensitivity/Recall): 0.4875
Specificity: 0.7947


## 6.3. LOGISTIC REGRESSION (LDA)

In [455]:

dt = LogisticRegression(random_state=0)
dt.fit(X_train_lda,y_train)

# Make predictions on the testing set
y_pred = dt.predict(X_test_lda)
print(set(y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = cm.ravel()

f1 = f1_score(y_test, y_pred)
print(f"F1 Skoru: {f1:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

precision = precision_score(y_test, y_pred)
print(f"Kesinlik (Precision): {precision:.4f}")

sensitivity = recall_score(y_test, y_pred)
print(f"Duyarlılık (Sensitivity/Recall): {sensitivity:.4f}")

specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

joblib.dump(dt, "lr_lda_model.pkl")
 



{np.int64(0), np.int64(1)}
F1 Skoru: 0.6104
Doğruluk (Accuracy): 0.7403
Kesinlik (Precision): 0.6351
Duyarlılık (Sensitivity/Recall): 0.5875
Specificity: 0.8212


['lr_lda_model.pkl']